In [9]:
import pandas as pd
import re
from collections import defaultdict
from langchain_core.runnables import RunnableLambda

def get_row_label(row):
    """대분류_소분류 형태의 row label 생성"""
    if pd.isna(row["소분류"]) or str(row["소분류"]).strip() == "":
        return str(row["대분류"]).strip()
    else:
        return f"{str(row['대분류']).strip()}_{str(row['소분류']).strip()}"

def load_survey_tables(file_path: str, sheet_name: str = "통계표"):
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    pattern = r"^[A-Z]+\d*[-.]?\d*\."
    question_indices = df[df[0].astype(str).str.match(pattern)].index.tolist()

    tables = {}
    question_texts = {}
    question_keys = []
    key_counts = defaultdict(int)

    for i, start in enumerate(question_indices):
        end = question_indices[i + 1] if i + 1 < len(question_indices) else len(df)
        title = str(df.iloc[start, 0]).strip()

        match = re.match(pattern, title)
        if not match:
            continue
        base_key = match.group().rstrip(".")
        key_counts[base_key] += 1
        suffix = f"_{key_counts[base_key]}" if key_counts[base_key] > 1 else ""
        final_key = base_key + suffix

        question_texts[final_key] = title + "(전체 단위 : %)"
        question_keys.append(final_key)

        table = df.iloc[start + 1:end].reset_index(drop=True)
        if len(table) >= 2:
            new_columns = table.iloc[0].fillna('').astype(str) + " " + table.iloc[1].fillna('').astype(str)
            new_columns.iloc[0] = "대분류"
            if len(new_columns) > 1:
                new_columns.iloc[1] = "소분류"
            table = table.drop([0, 1]).reset_index(drop=True)
            table.columns = new_columns
            table["대분류"] = table["대분류"].ffill()
            table = table.dropna(axis=1, how='all')
            table = table.dropna(axis=0, how='all')
            table = table.drop(index=0).reset_index(drop=True)
            if len(table) > 2:
                table = table.iloc[:-2].reset_index(drop=True)

            for col in table.columns:
                try:
                    numeric_col = pd.to_numeric(table[col], errors='coerce')
                    if numeric_col.notna().any():
                        table[col] = numeric_col.round(1)
                except:
                    continue

            tables[final_key] = table

    return tables, question_texts, question_keys

def linearize_row_wise(df):
    return " | ".join([
        f"{get_row_label(row)}; " + "; ".join(
            [f"{col}: {val}" for col, val in row.items() if col not in ["대분류", "소분류"]]
        )
        for _, row in df.iterrows()
    ])

def select_table(tables, question_keys, question_texts, index):
    if index.isdigit():
        choice = int(index) - 1
        if choice < 0 or choice >= len(question_keys):
            raise ValueError("올바른 질문 번호를 입력하세요.")
        selected_key = question_keys[choice]
    elif index in question_keys:
        selected_key = index
    else:
        raise ValueError("입력한 값이 올바른 질문 번호 또는 키가 아닙니다.")

    selected_table = tables[selected_key]
    selected_question = question_texts[selected_key]
    return selected_table, selected_question

def table_parser_node_fn(state):
    print("*" * 10, "Start table parsing", "*" * 10)
    file_path = state["file_path"]
    tables, question_texts, question_keys = load_survey_tables(file_path)

    print("\n📝 질문 목록:")
    for idx, key in enumerate(question_keys):
        print(f"{idx + 1}. [{key}] {question_texts[key]}")

    index = str(input("질문 index를 입력하세요: \n"))
    selected_table, selected_question = select_table(tables, question_keys, question_texts, index)
    linearized_table = linearize_row_wise(selected_table)

    state["selected_table"] = selected_table
    state["selected_question"] = selected_question
    state["linearized_table"] = linearized_table

    return state

table_parser_node = RunnableLambda(table_parser_node_fn)

# ✅ 테스트 코드
def test_table_selection():
    file_path = "/Users/jang-wonjun/Desktop/Dev/Tool_Agent_PCRP/agents/table_agents/table_list/서울시 대기환경 시민인식 조사.xlsx"
    state = {"file_path": file_path}
    result = table_parser_node.invoke(state)

    # print("\n✅ 선택된 질문:")
    # print(result["selected_question"])
    # print("\n✅ 선택된 테이블 (상위 5개 행만 출력):")
    # print(result["selected_table"].head())
    return result

result = test_table_selection()
result["selected_table"]

********** Start table parsing **********

📝 질문 목록:
1. [A1] A1. 귀하께서는 평소에 서울시 대기환경 문제에 대해 얼마나 관심이 있으십니까?(전체 단위 : %)
2. [A2] A2. 현재 서울시의 전반적인 대기오염 수준은 어느 정도라고 생각하십니까?(전체 단위 : %)
3. [A3] A3. 주요 대기오염물질별 서울시의 대기오염 수준은 어느 정도라고 생각하십니까? - 1) (초)미세먼지(전체 단위 : %)
4. [A3_2] A3. 주요 대기오염물질별 서울시의 대기오염 수준은 어느 정도라고 생각하십니까? -  2) 이산화질소(전체 단위 : %)
5. [A3_3] A3. 주요 대기오염물질별 서울시의 대기오염 수준은 어느 정도라고 생각하십니까? -   3) 오존(전체 단위 : %)
6. [A4] A4. 최근 3년 동안 서울시의 대기오염 수준이 과거(2022년 이전)보다 개선되었다고 생각하십니까?(전체 단위 : %)
7. [A4-1] A4-1. 서울시의 대기환경이 개선되었다고 생각하게 된 이유는 무엇입니까?(전체 단위 : %)
8. [A4-2] A4-2. 다양한 환경정책 가운데 어떤 정책이 대기환경 개선에 가장 많이 기여하였다고 생각하십니까?(전체 단위 : %)
9. [B1] B1. 이러한 서울시의 대기환경 개선 정책이 서울의 대기환경을 지금 보다 좋아지게 할 것이라고 생각하십니까?(전체 단위 : %)
10. [B1-1] B1-1. 그렇다면 대기환경 개선을 위해 어떤 정책이 가장 중요하다고 생각하십니까?(전체 단위 : %)
11. [B1-2] B1-2. 좋아지지 않을 것으로 생각하시는 이유는 무엇입니까?(전체 단위 : %)
12. [B2] B2. 서울시 대기환경 개선 사업 추진에서 가장 우선해야 할 부분은 무엇이라고 생각하십니까?(전체 단위 : %)
13. [B3] B3. 서울의 대기환경 개선을 위해 서울시가 가장 역점을 두고 추진해야 할 분야는 무엇입니까? (1순위)(전체 단위 : %)
14. [B3_2] B3. 서울의 

,대분류,소분류,사례수,대기환경 문제 관심정도 전혀 관심이 없다,대체로 관심이 없는 편,보통,대체로 관심이 있는 편,매우 관심 있다,관심없다 %,보통 %,관심있다 %,평균(5점척도)
0,전 체,NaN,1000,0.5,4.4,22.3,53.7,19.1,4.9,22.3,72.8,3.9
1,성별,남성,494,0.6,3.8,23.3,52.8,19.4,4.5,23.3,72.3,3.9
2,성별,여성,506,0.4,4.9,21.3,54.5,18.8,5.3,21.3,73.3,3.9
3,연령,20대,181,1.1,7.2,27.6,49.2,14.9,8.3,27.6,64.1,3.7
4,연령,30대,244,1.2,4.9,25.8,48.8,19.3,6.1,25.8,68.0,3.8
5,연령,40대,205,0.0,5.4,24.9,51.2,18.5,5.4,24.9,69.8,3.8
6,연령,50대,167,0.0,4.2,19.2,59.9,16.8,4.2,19.2,76.6,3.9
7,연령,60대이상,203,0.0,0.5,13.3,61.1,25.1,0.5,13.3,86.2,4.1
8,생활권역,도심권,51,5.9,0.0,19.6,51.0,23.5,5.9,19.6,74.5,3.9
9,생활권역,동북권,293,0.3,5.5,22.5,55.6,16.0,5.8,22.5,71.7,3.8


In [11]:
table = result["selected_table"]
table.head()

,대분류,소분류,사례수,대기환경 문제 관심정도 전혀 관심이 없다,대체로 관심이 없는 편,보통,대체로 관심이 있는 편,매우 관심 있다,관심없다 %,보통 %,관심있다 %,평균(5점척도)
0,전 체,NaN,1000,0.5,4.4,22.3,53.7,19.1,4.9,22.3,72.8,3.9
1,성별,남성,494,0.6,3.8,23.3,52.8,19.4,4.5,23.3,72.3,3.9
2,성별,여성,506,0.4,4.9,21.3,54.5,18.8,5.3,21.3,73.3,3.9
3,연령,20대,181,1.1,7.2,27.6,49.2,14.9,8.3,27.6,64.1,3.7
4,연령,30대,244,1.2,4.9,25.8,48.8,19.3,6.1,25.8,68.0,3.8


In [20]:
row_list = (table["대분류"] + "_" + table["소분류"]).dropna().tolist()
print(row_list)

['성별_남성', '성별_여성', '연령_20대', '연령_30대', '연령_40대', '연령_50대', '연령_60대이상', '생활권역_도심권', '생활권역_동북권', '생활권역_서북권', '생활권역_서남권', '생활권역_동남권', '가구원 수_1명', '가구원 수_2명', '가구원 수_3명', '가구원 수_4명이상', '환경취약계층_있다', '환경취약계층_없다', '기저질환 환자_있다', '기저질환 환자_없다', '주요 체류공간_실내', '주요 체류공간_실외', '대기오염 배출사업장_있다', '대기오염 배출사업장_없다']


In [17]:
table.columns

Index(['대분류', '소분류', '사례수 ', '대기환경 문제 관심정도 전혀 관심이 없다', ' 대체로 관심이 없는 편', ' 보통',
       ' 대체로 관심이 있는 편', ' 매우 관심 있다', '관심없다 %', '보통 %', '관심있다 %', '평균(5점척도) '],
      dtype='object')